In [13]:
import pandas as pd 
import re
import nltk
nltk.download('punkt')
import numpy as np
from tensorflow import keras

import tensorflow as tf
from keras.layers import Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.preprocessing.text import one_hot
from keras.layers import LSTM ,SpatialDropout1D
from tensorflow.keras.layers import Bidirectional
from keras.layers import Dense
from tensorflow.keras.layers import Dropout
from sklearn import preprocessing
from keras.callbacks import ModelCheckpoint


[nltk_data] Downloading package punkt to C:\Users\Muhammad
[nltk_data]     Swelam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import pandas as pd 
df = pd.read_csv("clean_data.csv")
df=df.dropna()
df.reset_index(inplace=True)
df.head()

,index,id,txt,dialect
0,0,1175358310087892992,بالنهاية ينتفض يغير,IQ
1,1,1175416117793349632,يعني محسوب البشر حيونه ووحشيه وتطلبون الغرب يح...,IQ
2,2,1175450108898565888,مبين كلامه خليجي,IQ
3,3,1175471073770573824,يسلملي مرورك وروحك الحلوه,IQ
4,4,1175496913145217024,وين الغيبه اخ محمد,IQ


In [3]:
X = df['txt']
y = df['dialect']
voc_size=8000


In [4]:
label_encoder = preprocessing.LabelEncoder()
y= label_encoder.fit_transform(y)
y

array([4, 4, 4, ..., 1, 1, 1])

In [5]:
X

0                                       بالنهاية ينتفض يغير
1         يعني محسوب البشر حيونه ووحشيه وتطلبون الغرب يح...
2                                          مبين كلامه خليجي
3                                 يسلملي مرورك وروحك الحلوه
4                                        وين الغيبه اخ محمد
                                ...                        
457959              لين صبحنا بوجهش زين سعاده مابعدها سعاده
457960                                  بالعافيه فقلي الاكل
457961                                              مدح يبي
457962    ايييي احبها واحب تحتها طاوله للكرك ويفضل بلك ل...
457963                      جيبي مطرقه شوفي اهدم واهدم ابوه
Name: txt, Length: 457964, dtype: object

In [6]:
corpus=[]
for i in X:
    corpus.append(i)
    

onehot_repr=[one_hot(words,voc_size)for words in corpus] 


In [7]:
sent_length=30
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3846   54 6998]
 [   0    0    0 ... 2674 7535 6771]
 [   0    0    0 ... 5472 1563 7267]
 ...
 [   0    0    0 ...    0 3189 1403]
 [   0    0    0 ... 2242 2441 2000]
 [   0    0    0 ... 7328 7014 1068]]


In [8]:
embedding_vector_features=50
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(18, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 30, 50)            400000    
                                                                 
 spatial_dropout1d (SpatialD  (None, 30, 50)           0         
 ropout1D)                                                       
                                                                 
 bidirectional (Bidirectiona  (None, 200)              120800    
 l)                                                              
                                                                 
 dense (Dense)               (None, 18)                3618      
                                                                 
Total params: 524,418
Trainable params: 524,418
Non-trainable params: 0
_________________________________________________________________
None


In [14]:
# Creat checkpoint to save model at the end of each epoch:
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5", save_best_only=True)

# Creat EarlyStopping checkpoint:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

In [15]:
X_final=np.array(embedded_docs)
y_final=np.array(y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2, random_state=42)


In [16]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=20,batch_size=64,callbacks=[checkpoint_cb, early_stopping_cb])


Epoch 1/20
5725/5725 [==============================] - 323s 56ms/step - loss: 2.2310 - accuracy: 0.2985 - val_loss: 2.0648 - val_accuracy: 0.3529
Epoch 2/20
5725/5725 [==============================] - 326s 57ms/step - loss: 2.0225 - accuracy: 0.3708 - val_loss: 1.9933 - val_accuracy: 0.3797
Epoch 3/20
5725/5725 [==============================] - 344s 60ms/step - loss: 1.9447 - accuracy: 0.3964 - val_loss: 1.9673 - val_accuracy: 0.3888
Epoch 4/20
5725/5725 [==============================] - 336s 59ms/step - loss: 1.8953 - accuracy: 0.4121 - val_loss: 1.9524 - val_accuracy: 0.3945
Epoch 5/20
5725/5725 [==============================] - 335s 59ms/step - loss: 1.8618 - accuracy: 0.4227 - val_loss: 1.9552 - val_accuracy: 0.3961
Epoch 6/20
5725/5725 [==============================] - 339s 59ms/step - loss: 1.8340 - accuracy: 0.4317 - val_loss: 1.9462 - val_accuracy: 0.3984
Epoch 7/20
5725/5725 [==============================] - 334s 58ms/step - loss: 1.8116 - accuracy: 0.4378 - val_loss: 1

In [17]:
y_pred=model.predict(X_test)
y_pred=np.argmax(y_pred,axis=1)

y_pred=label_encoder.inverse_transform(y_pred)
y_test=label_encoder.inverse_transform(y_test)


from sklearn.metrics import confusion_matrix ,classification_report
from sklearn.metrics import accuracy_score

accuracy_score(y_test,y_pred)

0.3984474796108873

In [18]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          AE       0.34      0.23      0.27      5379
          BH       0.26      0.17      0.21      5216
          DZ       0.49      0.37      0.42      3337
          EG       0.55      0.76      0.64     11462
          IQ       0.45      0.39      0.42      3048
          JO       0.34      0.17      0.23      5731
          KW       0.33      0.46      0.39      8337
          LB       0.48      0.48      0.48      5378
          LY       0.39      0.53      0.45      7356
          MA       0.52      0.47      0.49      2297
          OM       0.25      0.15      0.19      3823
          PL       0.34      0.46      0.39      8622
          QA       0.36      0.35      0.36      6204
          SA       0.26      0.28      0.27      5394
          SD       0.53      0.41      0.46      2929
          SY       0.33      0.25      0.28      3201
          TN       0.49      0.27      0.35      1858
          YE       0.35    